In [0]:
pip install fuzzywuzzy

Python interpreter will be restarted.
Requirement already satisfied: fuzzywuzzy in /local_disk0/.ephemeral_nfs/envs/pythonEnv-dab056ce-bd65-4355-a020-d572367442fc/lib/python3.8/site-packages (0.18.0)
Python interpreter will be restarted.

In [0]:
pip install openpyxl

Python interpreter will be restarted.
Requirement already satisfied: openpyxl in /local_disk0/.ephemeral_nfs/envs/pythonEnv-dab056ce-bd65-4355-a020-d572367442fc/lib/python3.8/site-packages (3.0.10)
Requirement already satisfied: et-xmlfile in /local_disk0/.ephemeral_nfs/envs/pythonEnv-dab056ce-bd65-4355-a020-d572367442fc/lib/python3.8/site-packages (from openpyxl) (1.1.0)
Python interpreter will be restarted.

In [0]:
import pandas as pd
import numpy as np
import re
import string

import pickle
import matplotlib.pyplot as plt
import nltk
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem  import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from fuzzywuzzy import fuzz

from datetime import datetime
from decimal import Decimal


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Package wordnet is already up-to-date!
/local_disk0/.ephemeral_nfs/envs/pythonEnv-dab056ce-bd65-4355-a020-d572367442fc/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
 warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')

In [0]:
###Load STG1 & STG2 Model Result
stg1_model_src = '/dbfs/FileStore/shared_uploads/xiaomeng.fan@aia.com/POS_MAIA_Questions_Classification_Model.sav'
stg2_model_src = '/dbfs/FileStore/shared_uploads/xiaomeng.fan@aia.com/POS_MAIA_Answer_Examination_Model.sav'

stg1_model = pickle.load(open(stg1_model_src,'rb'))
stg2_model = pickle.load(open(stg2_model_src,'rb'))

In [0]:
### Query Last Month Data
query_batch_raw = spark.sql("""
select distinct question, prediction as response, probability as confidence from hive_metastore.ops_aos_read.maia_training
where
month(current_date())-month(createdDate) = 1
and length(user_id) = 5
and appSource = 'AGIA'
and role in ('AGY', 'AFA')
-- filter not null responses
and probability is not null
""")

In [0]:
df_batch_raw = query_batch_raw.toPandas()

/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [confidence] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(

In [0]:
### Load MAIA Stopwords + NLTK Stopwords
stopwords_query_result = spark.sql("""
select * from ops_pos_wrk.aosda_maia_stopwords
""")

df_stopwords = stopwords_query_result.toPandas()
maia_sw_list = df_stopwords['stopword'].tolist()

stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.extend(maia_sw_list)

Stage 1

In [0]:
### Stg1 Data Pre-processing
## Load POS Unique Term
file_path = "/dbfs/FileStore/shared_uploads/xiaomeng.fan@aia.com/"
df_pos_term = pd.read_excel(file_path+"pos_terms.xlsx")
pos_term_list = "|".join([i.lower() for i in df_pos_term['pos_terms']])

In [0]:
def qns_preprocessing(text):
  lemmatizer = WordNetLemmatizer()
  pol_format = re.compile(r'\w{1}\d{9}')
  nric_format = re.compile(r'[sfgtm]\d{7}[a-z]')
  emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
  
  text_orig = text.lower()
  text_pol_clean = pol_format.sub('AIAPolNum',text_orig)
  text_pol_nric_clean = nric_format.sub('CustNRIC',text_pol_clean)
  text_pol_nric_posterm_clean = re.sub(pos_term_list, 'AIAPOSTerm', text_pol_nric_clean)
  text_pol_nric_posterm_dpmt_clean = re.sub(r'cnc|pos|u&i|clm','OpsDpmt', text_pol_nric_posterm_clean)
  text_pol_nric_posterm_dpmt_sw_clean = " ".join([word for word in nltk.word_tokenize(text_pol_nric_posterm_dpmt_clean) if word not in stopword_list])
  text_pol_nric_posterm_dpmt_sw_punc_clean = " ".join(word.strip(string.punctuation) for word in text_pol_nric_posterm_dpmt_sw_clean.split())
  text_pol_nric_posterm_dpmt_sw_punc_lem_clean = " ".join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text_pol_nric_posterm_dpmt_sw_punc_clean)]) ## 1. why some have [] some dont  2. why token not split()
  
  return text_pol_nric_posterm_dpmt_sw_punc_lem_clean

df_batch_raw['qns_clean'] = df_batch_raw['question'].apply(qns_preprocessing)

In [0]:
###Load the Stg1 original cleaned data
df_stg1_training = pd.read_pickle(file_path+"stg1_clean.pkl")
display(df_stg1_training)

qns,tag,qns_clean
H223497209 giro,CNC,AIAPolNum giro
Thanks thats all,Small Talk,thats
Q514050177,Small Talk,AIAPolNum
Q514050177 dividend,POS,AIAPolNum dividend
Q514050177 dividend payment,POS,AIAPolNum dividend payment
L533278532 coupon,POS,AIAPolNum coupon
Q514050177 dividend withdrawal,POS,AIAPolNum dividend withdrawal
premium holiday,POS,premium holiday
replacement,Small Talk,replacement
yes,Small Talk,


In [0]:
###################CSV might changed the datatype, tfidf every time different

seed_num = '123'

tfidf = TfidfVectorizer(min_df = 0.002, max_df = 0.9, ngram_range = (1,2), use_idf = True, sublinear_tf = True)
Fit = tfidf.fit(df_stg1_training['qns_clean'])
features_stg1 = tfidf.transform(df_batch_raw.qns_clean).toarray()

pred_stg1 = stg1_model.predict(features_stg1)

In [0]:
df_batch_raw['department_predict'] = pred_stg1

In [0]:
df_batch_raw[['batch_dt','batch_index']] = datetime.now() , df_batch_raw.index
df_batch_raw['batch_key'] = df_batch_raw['batch_dt'].astype(str)+'-'+df_batch_raw['batch_index'].astype(str)

/databricks/python/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
 return array(a, dtype, copy=False, order=order)

In [0]:
df_batch_stg1_final = df_batch_raw[['batch_key','question','response','confidence','qns_clean','department_predict']]

In [0]:
display(df_batch_stg1_final)

batch_key,question,response,confidence,qns_clean,department_predict
2022-09-17 17:32:23.323162-0,where can i find the medical history form?,Forms,0.36,find medical history form,Non POS
2022-09-17 17:32:23.323162-1,Can I cancel the termination request for L538666424?,Withdrawal_Submission,0.51,cancel termination request AIAPolNum,POS
2022-09-17 17:32:23.323162-2,Policy loan,Loan_Application,0.88,policy loan,POS
2022-09-17 17:32:23.323162-3,questions for policy H235750471 and policy P560526956,Agency_Policy_Tagging,0.29,question policy AIAPolNum policy AIAPolNum,Non POS
2022-09-17 17:32:23.323162-4,ca,ST_gibberish,0.59,ca,Non POS
2022-09-17 17:32:23.323162-5,What is the charges for invest easy using cash,POS_ILP_Penalty_Charges,0.50,charge invest easy using cash,Non POS
2022-09-17 17:32:23.323162-6,H240932750 health shield upgrade,Changes_to_plan,0.70,AIAPolNum health shield upgrade,POS
2022-09-17 17:32:23.323162-7,Check,POS_Policy_information,0.41,,Non POS
2022-09-17 17:32:23.323162-8,Initiate live chat,Help,0.57,initiate,Non POS
2022-09-17 17:32:23.323162-9,witness,ST_evil,0.47,witness,Non POS


In [0]:
#%sql
#drop table ops_aos_wrk.maia_question_batch_classification;
#create table ops_aos_wrk.maia_question_batch_classification
#(
#batch_key String,
#question String,
#response String,
#confidence Decimal(3,2),
#qns_clean String,
#department_predict String
#)

In [0]:
spark_df_stg1 = spark.createDataFrame(df_batch_stg1_final)
spark_df_stg1.write.mode('append').saveAsTable('ops_aos_wrk.maia_question_batch_classification')

Stage 2

In [0]:
df_watson, df_stg2_raw = spark.sql("""select * from ops_pos_wrk.watson_list""").toPandas(), spark.sql("""select * from ops_aos_wrk.maia_question_batch_classification where department_predict = 'POS' """).toPandas()

/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [confidence] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(

In [0]:
df_watson['Intent_list'] = df_watson['Intent_Conditions'].apply(lambda intent: intent.strip(string.punctuation))

In [0]:
df_stg2_watson = df_stg2_raw.merge(df_watson['Intent_list'], how='left', left_on='response',right_on='Intent_list')

In [0]:
df_stg2_watson['Intent_in_watson_POS'] = np.where(df_stg2_watson['Intent_list'].isnull(), '0','1')

In [0]:
def stg2_preprocess_1(intent):
  intent_raw = intent.lower()
  intent_punc = " ".join([text.strip(string.punctuation) for text in intent_raw.split('_')])
  intent_punc_sw = " ".join([text for text in nltk.word_tokenize(intent_punc) if text not in stopword_list])
  
  return intent_punc_sw  

In [0]:
def stg2_preprocess_2(question):
  lemmatizer = WordNetLemmatizer()
  pol_format = re.compile(r'\w{1}\d{9}')
  nric_format = re.compile(r'[sfgtm]\d{7}[a-z]')
  emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
  
  qns_raw = question.lower()
  qns_pol = pol_format.sub(r'', qns_raw)
  qns_pol_nric = nric_format.sub(r'', qns_pol)
  qns_pol_nric_emoji = emoji_pattern.sub(r'', qns_pol_nric)
  qns_pol_nric_emoji_punc = " ".join([text.strip(string.punctuation) for text in qns_pol_nric_emoji.split(' ')])
  qns_pol_nric_emoji_punc_sw = " ".join([text for text in nltk.word_tokenize(qns_pol_nric_emoji_punc) if text not in stopword_list])
  qns_pol_nric_emoji_punc_sw_lm = ' '.join([lemmatizer.lemmatize(text) for text in nltk.word_tokenize(qns_pol_nric_emoji_punc_sw)])
  
  return qns_pol_nric_emoji_punc_sw_lm

In [0]:
df_stg2_watson['response_c'] = df_stg2_watson['response'].apply(stg2_preprocess_1)
df_stg2_watson['qns_c'] = df_stg2_watson['question'].apply(stg2_preprocess_2)
df_stg2_watson['response_qns_c_combine'] = df_stg2_watson.apply(lambda df: df['response_c'] + ' ' + df['qns_c'], axis = 1)

In [0]:
## load stg2 training set
df_stg2_training = pd.read_pickle(file_path+"stg2_clean.pkl")
display(df_stg2_training)

response,question,confidence,tag,Department,is_pos_content,response_c,question_c,Ratio,Ratio_combine,response_question,row_similarities
Refund,Q514050177 dividend payment,0.32420958280563356,No,POS,1,refund,dividend payment,0.27,0.08753658735752107,refund dividend payment,0.0
Coupon_Payment_Details,L533278532 coupon,0.7661356449127197,Yes,POS,1,coupon payment details,coupon,0.43,0.32943832731246947,coupon payment details coupon,0.5957817822877031
Withdrawal_Procedures,Q514050177 dividend withdrawal,0.9163254261016847,Yes,POS,1,withdrawal procedures,dividend withdrawal,0.5,0.45816271305084233,withdrawal procedures dividend withdrawal,0.44429209974432626
Premium_Holiday,premium holiday,0.9836024284362794,Yes,POS,1,premium holiday,premium holiday,1.0,0.9836024284362794,premium holiday premium holiday,1.0
Reinstatement_Procedures_Rules,H235938743 reinstatement,0.5169537782669067,Yes,POS,1,reinstatement procedures rules,reinstatement,0.6,0.310172266960144,reinstatement procedures rules reinstatement,0.5433500159506385
NB_Instant_COL,"H224151117 - client signed and sent in a counteroffer, why counter offer closed?",0.6671443939208985,No,POS,0,nb instant col,"- client signed and sent in a counteroffer , why counter offer closed ?",0.21,0.14010032272338868,"nb instant col - client signed and sent in a counteroffer , why counter offer closed ?",0.0
POS_LPA_POA,"Q512134642 - Power of Attorney (LPA) form 1 done, can the donee can sign on all policy changes from now on?",0.9699352741241456,Yes,POS,1,pos lpa poa,"- power of attorney ( lpa ) form 1 done , can the donee can sign on all policy change from now on ?",0.18,0.1745883493423462,"pos lpa poa - power of attorney ( lpa ) form 1 done , can the donee can sign on all policy change from now on ?",0.24773678864776605
Marketing_SFT,"good afternoon, may i check if i can add in rider for an older term ""AIA Secure Term"", lets say Critical Illness rider",0.7572103023529053,No,POS,0,marketing sft,"good afternoon , may i check if i can add in rider for an older term `` aia secure term '' , let say critical illness rider",0.12,0.09086523628234863,"marketing sft good afternoon , may i check if i can add in rider for an older term `` aia secure term '' , let say critical illness rider",0.0
Add_Rider,add in rider,0.8875681400299074,Yes,POS,1,add rider,add in rider,0.86,0.7633086004257204,add rider add in rider,0.8311766594166912
Freelook,L548630136 - cancel policy with free look,0.7700406074523927,Yes,POS,1,freelook,- cancel policy with free look,0.42,0.3234170551300049,freelook - cancel policy with free look,0.0


In [0]:
tfidf = TfidfVectorizer(ngram_range = (1,1))
tfidf.fit(df_stg2_training['response_question'])
tfidf_qns = tfidf.transform(df_stg2_watson['qns_c']).todense()
tfidf_response = tfidf.transform(df_stg2_watson['response_c']).todense()

row_similarities = [1-spatial.distance.cosine(tfidf_qns[x],tfidf_response[x]) for x in range(len(tfidf_qns))]

df_stg2_watson['row_similarities'] = row_similarities

/databricks/python/lib/python3.8/site-packages/scipy/spatial/distance.py:728: RuntimeWarning: invalid value encountered in double_scalars
 dist = 1.0 - uv / np.sqrt(uu * vv)

In [0]:
df_stg2_watson['ratio'] =  df_stg2_watson.apply(lambda x: Decimal(fuzz.ratio(x['qns_c'],x['response_c'])*0.01), axis = 1)
df_stg2_watson['ratio_combine'] = df_stg2_watson['ratio']*df_stg2_watson['confidence']

In [0]:
df_stg2_batch = df_stg2_watson[['ratio', 'ratio_combine', 'confidence', 'Intent_in_watson_POS', 'row_similarities']]

for i in df_stg2_batch.columns:
  df_stg2_batch[i] = df_stg2_batch[i].apply(lambda x : float(x))

<command-2829380880215015>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 df_stg2_batch[i] = df_stg2_batch[i].apply(lambda x : float(x))

In [0]:
df_stg2_watson['MAIA_Answer_Correct'] = stg2_model.predict(df_stg2_batch)
df_stg2_upload = df_stg2_watson[['batch_key','question','response','MAIA_Answer_Correct']]

In [0]:
#%sql

#create table ops_aos_wrk.maia_question_batch_answer_examination
#(
#batch_key String,
#question String,
#response String,
#MAIA_Answer_Correct String
#)

In [0]:
spark_df_stg2_upload = spark.createDataFrame(df_stg2_upload)
spark_df_stg2_upload.write.mode('append').saveAsTable('ops_aos_wrk.maia_question_batch_answer_examination')